# flavio tutorial

## Part 2: Basic concepts



## Observables, parameters, predictions

- An *observable* $O$ in flavio is defined as a scalar physical quantity that can be *predicted* as a function of *parameters* $\vec\theta$ and *Wilson coefficients* $\vec C$ and then compared to experiment

$$P_O: \vec\theta, \vec C \to \mathbb R$$

- Example: observable $O=\text{BR}(B_s\to\mu^+\mu^-)$, parameters $\vec\theta = (G_F, f_{B_s}, \ldots)$, Wilson coefficients $\vec C = (C_{10}, \ldots)$
- New physics only affects Wilson coefficients (more later)


## More on parameters

- Real numbers that can be measured quantities (masses, lifetimes, ...) or theoretical parameters (form factor parametrization coefficients, ...)
- Identified by string name, e.g. `'m_mu'`, `'tau_Bs'`
- See <https://flav-io.github.io/docs/parameters.html> for full list
- Default values set on import but can be easily modified

Parameters correspond to instances of the `flavio.Parameter` class and can be accessed via their name

We can get a dictionary with all parameters by showing the instances of the class

In [ ]:
import flavio

In [ ]:
flavio.Parameter.instances

We can use the `find()` method of the `flavio.Parameter` class to find parameters

In [ ]:
flavio.Parameter.find('m_') # find mass parameters

### Pretty representation of paramers in Jupyter

Accessing a parameter via its name shows LaTeX name and description

In [ ]:
flavio.Parameter['GF']

But be careful since the following defines a new instance and overwrites the exsisting one:

In [ ]:
flavio.Parameter('GF')

We can read in the default parameters again using the `read_default()` method of `flavio.default_parameters`

In [ ]:
flavio.default_parameters.read_default()

In [ ]:
flavio.Parameter['GF']

Another useful method of `flavio.default_parameters` is `get_central_all()`. As the name suggests, it returns all central values of the default parameters

In [ ]:
flavio.default_parameters.get_central_all()

> Q: So what's the difference between a parameter and a measurement?  

A: For flavio, a *parameter* is used as an input to a (theory) prediction of an *observable*, while a *measurement* is an experimental determination of this *observable* (more later)

## More on observables

- Observables are identified by string names as well, e.g. `'BR(Bs->mumu)'`
- See https://flav-io.github.io/docs/observables.html for full list
- Observables have (theory) **prediction** and **measurement** (or several) associated to them

Observables correspond to instances of the `flavio.Observable` class and can be accessed via their name

We can get a dictionary with all observables by showing the instances of the class

In [ ]:
flavio.Observable.instances

We can use the `find()` method of the `flavio.Observable` class to find observables using regular expressions

In [ ]:
flavio.Observable.find('^BR.*->mumu\)')

### Pretty representation of observables in Jupyter

Accessing an observables via its name shows LaTeX name, description, and function for theory prediction

In [ ]:
flavio.Observable['BR(Bs->mumu)']

Be carefule to not use `flavio.Observable('BR(Bs->mumu)')`, which defines a new instance and overwrites the existing one!

## SM prediction of an observable

In [ ]:
flavio.sm_prediction('BR(Bs->mumu)')

## Observables with arguments

- Observables can also have additional arguements
- Example: branching ratios differential in $q^2$
- Numerical value of argument must be provided when computing prediction

In [ ]:
flavio.sm_prediction('dBR/dq2(B0->Denu)', q2=3)

Accessing an observables with arguments via its name also shows the required arguments

In [ ]:
flavio.Observable['dBR/dq2(B0->Denu)']

... what `sm_prediction` actually does is to compute the *prediction* for *central values* of all *parameters*

In [ ]:
flavio.sm_prediction('BR(Bs->mumu)')

In [ ]:
WCs_SM = flavio.WilsonCoefficients()
pars_central = flavio.default_parameters.get_central_all()

obs = flavio.Observable['BR(Bs->mumu)']
obs.prediction.function(WCs_SM, pars_central)

## Theoretical uncertainties

- theory uncertainty = probability (or credibility) distribution associated with a prediction
- sources: parametric, higher-order, systematic
- In flavio, **all theory uncertainties are parametric**: PDFs associated with parameters

## Probability distributions in flavio

`flavio.statistics.probability` provides a number of one- and multidimensional PDFs:

In [ ]:
[c.__name__ for c
 in flavio.statistics.probability.ProbabilityDistribution.get_subclasses()]

## Defining parameter uncertainties

- The `flavio.classes.ParameterConstraints` class contains the PDFs associated with parameters
- There is a default instance used by convenience functions like `sm_prediction`: `flavio.default_parameters`
- Default parameters are populated from two YAML files:
    - [parameters_uncorrelated.yml](https://github.com/flav-io/flavio/blob/master/flavio/data/parameters_uncorrelated.yml)
    - [parameters_correlated.yml](https://github.com/flav-io/flavio/blob/master/flavio/data/parameters_correlated.yml)

Example: set bottom quark mass & uncertainty

In [ ]:
par = flavio.default_parameters
from flavio.statistics.probability import NormalDistribution

# verbose version
par.add_constraint(
    parameters=['m_b'],
    constraint=NormalDistribution(central_value=4.18, standard_deviation=0.04)
)

# short version
par.set_constraint('m_b', '4.18 +- 0.04')

par.get_central('m_b'), par.get_1d_errors_rightleft()['m_b']

In [ ]:
par.read_default()
par.get_central('m_b'), par.get_1d_errors_rightleft()['m_b']

## Computing theory uncertainties

The default method to compute theory uncertainties is to generate $N$ random values for all *parameters*, compute the prediction for each random set, and take the mean/variance.

NB, the relative precision is $1/\sqrt{2N}$, computing time $\propto N$.

In [ ]:
print('central: ',             flavio.sm_prediction('BR(Bs->mumu)'))
print('uncertainty, N=10: ',   flavio.sm_uncertainty('BR(Bs->mumu)', N=10))
print('uncertainty, N=100: ',  flavio.sm_uncertainty('BR(Bs->mumu)', N=100))
print('uncertainty, N=1000: ', flavio.sm_uncertainty('BR(Bs->mumu)', N=1000))
print('uncertainty, N=5000: ', flavio.sm_uncertainty('BR(Bs->mumu)', N=5000))

NB, the default value `N=100` is usually a good compromise between speed & accuracy, but for the final numbers in your paper, better crank it up to 500 or so

## Parallel processing for uncertainty calculation

- Uncertainty calculation can be very slow for some observables
- Speed it up by using multiple cores

In [ ]:
%%time
flavio.sm_uncertainty('<P5p>(B0->K*mumu)',
                      q2min=1, q2max=6, N=100)

In [ ]:
%%time
flavio.sm_uncertainty('<P5p>(B0->K*mumu)',
                      q2min=1, q2max=6, N=100, threads=8)

### Visualizing error budgets

To understand the dominant sources of theory uncertainties, it can be useful to evaluate the uncertainties for one parameter (or correlated set of parameters) at a time. In flavio, this is done with the function `sm_error_budget`:

In [ ]:
err_bsmumu = flavio.sm_error_budget('BR(Bs->mumu)')
err_bsmumu

This can now be visualized.

Necessary imports:

In [ ]:
import flavio.plots as fpl
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
fpl.error_budget_pie(err_bsmumu);

Correlated uncertainties are lumped together:

In [ ]:
err_p5p = flavio.sm_error_budget('P5p(B0->K*mumu)', q2=4)
fpl.error_budget_pie(err_p5p);

### Error budgets: caveat

The implementations in flavio are a compromise between accuracy and computational speed. For statistical inference, only the total uncertainty is relevant. Thus, subleading contributions that are much smaller than the dominant uncertainty or much smaller than current experimental uncertainties are sometimes omitted.

## Measurements

- A *measurement* is a *probability distribution* associated with one (or more) *observable(s)*
- This probability distribution is provided by experiments and included in `flavio`

## Predefined measurements

- Contained in [measurements.yaml](https://github.com/flav-io/flavio/blob/master/flavio/data/measurements.yml)
- Simple example:

```yaml
LHCb rad 2012:                       # name
  experiment: 'LHCb'                 # experiment
  inspire: 'Aaij:2012ita'            # tex key
  values:
    BR(Bs->phigamma): 3.5 ± 0.4 e-5  # constraint
```

The constraint associates a normal distribution with the observable

Measurements correspond to instances of the `flavio.Measurement` class and can be accessed via their name

We can get a dictionary with all measurements by showing the instances of the class

In [ ]:
flavio.Measurement.instances

We can use the `find()` method of the `flavio.Measurement` class to find measurements

In [ ]:
flavio.Measurement.find('Belle')

### Getting measurements of a given observable

The `get_measurements()` method of the `flavio.Observable` class lists all measurements of a given observable instance

In [ ]:
flavio.Observable['<P5p>(B0->K*mumu)'].get_measurements()

### Pretty representation of measurements in Jupyter

Accessing an measurement via its name shows experiment name, a link to Inspire, and the measured observables

In [ ]:
flavio.Measurement['LHCb B->K*mumu 2020 P 4-6']

Again, note that `flavio.Measurement('LHCb B->K*mumu 2020 P 6-8')` defines a new instance and overwrites the existing one!

But it is easy to load the default measurements again

In [ ]:
flavio.measurements.read_default()

### Things to do with measurements: e.g. plots

In [ ]:
import matplotlib.pyplot as plt
obs = '<P5p>(B0->K*mumu)';
fpl.bin_plot_exp(obs, col_dict={'LHCb': 'C0', 'CMS': 'C1', 'ATLAS': 'C2'});
plt.legend()
plt.xlabel(r'$q^2$ [GeV$^2$]')
plt.ylabel(flavio.Observable[obs].tex);

Plotting the theory prediction with uncertatinties

In [ ]:
obs = 'P5p(B0->K*mumu)';
fpl.diff_plot_th_err(obs, 0.1, 6, threads=8, label=r'low $q^2$');
fpl.diff_plot_th_err(obs, 15, 19, threads=8, label=r'high $q^2$');
plt.legend()
plt.xlabel(r'$q^2$ [GeV$^2$]')
plt.ylabel(flavio.Observable[obs].tex);

## Exercise

Make a plot comparing the LHCb measurement of $P_5'$ in $B\to K^*\mu^+\mu^-$ to the *binned* SM prediction.

Hints:
- use `flavio.plots.bin_plot_th` for the latter
- Check the `flavio.plots` API docs: <https://flav-io.github.io/apidoc/flavio/plots/plotfunctions.m.html>

Next: <a href="3 Wilson coefficients.ipynb">Wilson coefficients</a>